In [ ]:
%%HTML
<style>
.container { width: 100% }
</style>

# From Regular Expressions to <span style="font-variant:small-caps;">Fsm</span>s

In [ ]:
class RegExp2NFA:
    def __init__(self, Sigma):
        self.Sigma       = Sigma
        self.StateCount = 0

In [ ]:
def toNFA(self, r):
    if r == 0: 
        return self.genEmptyNFA()
    if r == '': 
        return self.genEpsilonNFA()
    if isinstance(r, str) and len(r) == 1: 
        return self.genCharNFA(r)
    if r[0] == 'cat':
        return self.catenate(self.toNFA(r[1]), self.toNFA(r[2]))
    if r[0] == 'or':
        return self.disjunction(self.toNFA(r[1]), self.toNFA(r[2]))
    if r[0] == 'star':
        return self.kleene(self.toNFA(r[1]))
    raise ValueError(f'{r} is not a proper regular expression.')
    
RegExp2NFA.toNFA = toNFA
del toNFA

In [ ]:
def genEmptyNFA(self):
        q0 = self.getNewState()
        q1 = self.getNewState()
        delta = {}
        return {q0, q1}, self.Sigma, delta, q0, { q1 }

RegExp2NFA.genEmptyNFA = genEmptyNFA
del genEmptyNFA

In [ ]:
def genEpsilonNFA(self):
    q0 = self.getNewState()
    q1 = self.getNewState()
    delta = { (q0, ''): {q1} }
    return {q0, q1}, self.Sigma, delta, q0, { q1 }

RegExp2NFA.genEpsilonNFA = genEpsilonNFA 
del genEpsilonNFA

In [ ]:
def genCharNFA(self, c):
    q0 = self.getNewState()
    q1 = self.getNewState()
    delta = { (q0, c): {q1} } 
    return {q0, q1}, self.Sigma, delta, q0, { q1 }

RegExp2NFA.genCharNFA = genCharNFA
del genCharNFA

In [ ]:
def catenate(self, f1, f2):
        M1, Sigma, delta1, q1, A1 = f1
        M2, Sigma, delta2, q3, A2 = f2
        q2 = arb(A1)
        q4 = arb(A2)
        delta = dict_union(delta1, delta2)
        delta[q2, ''] = {q3}
        return M1 | M2, Sigma, delta, q1, A2

RegExp2NFA.catenate = catenate
del catenate

In [ ]:
def disjunction(self, f1, f2):
        M1, Sigma, delta1, q1, A1 = f1
        M2, Sigma, delta2, q2, A2 = f2
        q3 = arb(A1)
        q4 = arb(A2)
        q0 = self.getNewState()
        q5 = self.getNewState() 
        delta = dict_union(delta1, delta2)
        delta[q0, ''] = { q1, q2 }
        delta[q3, ''] = { q5 }
        delta[q4, ''] = { q5 }
        return { q0, q5 } | M1 | M2, Sigma, delta, q0, { q5 }
    
RegExp2NFA.disjunction = disjunction
del disjunction

In [ ]:
def kleene(self, f):
    M, Sigma, delta0, q1, A = f
    q2 = arb(A)
    q0 = self.getNewState()
    q3 = self.getNewState()
    delta = delta0
    delta[q0, ''] = { q1, q3 }
    delta[q2, ''] = { q1, q3 }
    return { q0, q3 } | M, Sigma, delta, q0, { q3 }

RegExp2NFA.kleene = kleene
del kleene

In [ ]:
def getNewState(self):
    self.StateCount += 1
    return self.StateCount

RegExp2NFA.getNewState = getNewState
del getNewState

In [ ]:
def arb(S):
    for x in S:
        return x

In [ ]:
def dict_union(d1, d2):
    return { **d1, **d2 }

In [ ]:
A = { 'a': 1, 'c': 3 }
B = { 'b': 2, 'c': 4 }
dict_union(A, B)

In [ ]:
A, B